<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ
</h1>

---

<h1 style="text-align: center;">
    Convolution and pooling operations
</h1>

## Convolution (свёртка)

На этом семинаре мы посмотрим, как свёртки влияют на изображение и попрактикуемся в вычислении свёрток и пулингов от различных изображений.

Для начала напомним, что такое свёртка:

<img src="../img/convnet_conv_pool_1.gif" width=500>

То есть мы берём фильтр размера $F \times F$, умножаем его на область изображения размером $F \times F$ поэлементно, складываем получившиеся поэлемнетные произведения и записываем это число в результирующий тензор.

За исключением архитектур типа **MobileNet**, третья размерность фильтра всегда совпадает с третьей размерностью входного тензора. Если картинка размера $H \times W \times 3$, то фильтр будет иметь размер $F \times F \times 3$, и поэлементное произведение будет производиться по всему объёму.

Напомним два важных параметра операции свертки:

* **stride** - это размер шага окна свёртки по осям $x$ и $y$ (обычно совпадают, но вполне могут быть и разными).

* **padding** - это окружение картинки по краям нулями (или чем-то другим) для того, чтобы придать изображению после свёртки нужный размер (пэддинг делается до свёртки)

---

Посмотрим на то, как применение свёртки с определёнными фильтрами влияет на изображение. На этот счёт есть [хорошая статья на Хабре](https://habr.com/post/142818/). 

Возьмём код из статьи и посмотрим, как будет меняться картинка в зависимости от фильтра:

In [ ]:
from PIL import Image
from math import ceil, sqrt
from math import floor

import matplotlib.pyplot as plt
%matplotlib inline


def checkByte(a):
    if a > 255:
        a = 255
    elif a < 0:
        a = 0
    return a


def conv(a, b):
    res = 0
    
    for i in range(len(a)):
        for j in range(len(a[0])):
            res += a[i][j] * b[i][j]

    return res
  

def median(a):
    flat = []
    
    for i in range(len(a)):
        for j in range(len(a[0])):
            flat.append(a[i][j])
            
    flat.sort()
    
    return flat[ceil(len(flat) / 2)]
  

def max(a):
    flat = []
    
    for i in range(len(a)):
        for j in range(len(a[0])):
            flat.append(a[i][j])
            
    flat.sort()
    
    return flat[len(flat) - 1]
  

def min(a):
    flat = []
    
    for i in range(len(a)):
        for j in range(len(a[0])):
            flat.append(a[i][j])

    flat.sort()
    
    return flat[0]


im = Image.open('data/lenna.jpg')

pixels = im.load()

plt.imshow(im)
plt.show()

In [ ]:
imFinal = im.copy()
pixels2 = imFinal.load()


filter = [
    [-1, -1,  0,  0, 0],
    [ 0, -1, -1, -1, 0],
    [ 0, -1,  9, -1, 0],
    [ 0, -1, -1, -1, 0],
    [ 0,  0,  0,  0, 0],
]


# filter = [
#     [-1, -1, -1, -1, -1],
#     [-1, -1, -1, -1, -1],
#     [-1, -1,  4, -1, -1],
#     [-1, -1, -1, -1, -1],
#     [-1, -1, -1, -1, -1],
# ]


# filter = [
#     [0, 0, 0, 1, 0, 0, 0],
#     [0, 0, 1, 1, 1, 0, 0],
#     [0, 1, 1, 1, 1, 1, 0],
#     [1, 1, 1, 1, 1, 1, 1],
#     [0, 1, 1, 1, 1, 1, 0],
#     [0, 0, 1, 1, 1, 0, 0],
#     [0, 0, 0, 1, 0, 0, 0],
# ]


# filter = [
#     [-1, -1, -1],
#     [-1,  9, -1],
#     [-1, -1, -1],
# ]


# filter = [
#     [0.5, 1.5,  2, 1.5, 0.5],
#     [1.5, 3.5,  5, 3.5, 1.5],
#     [  2,   5, 10,   5,   2],
#     [1.5, 3.5,  5, 3.5, 1.5],
#     [0.5, 1.5,  2, 1.5, 0.5],
# ]


div = 0

for i in range(len(filter)):
    for j in range(len(filter[0])):
        div += filter[i][j]
        
if div == 0:
    div = 1

for i in range(floor(len(filter) / 2), im.width - floor(len(filter) / 2)):
    for j in range(floor(len(filter) / 2), im.height - floor(len(filter) / 2)):
        matr_r = []
        matr_g = []
        matr_b = []
        for n in range(-floor(len(filter) / 2), ceil(len(filter) / 2)):
            row_r = []
            row_g = []
            row_b = []
            for m in range(-floor(len(filter) / 2), ceil(len(filter) / 2)):
                r, g, b = pixels[i + n, j + m]
                row_r.append(r)
                row_g.append(g)
                row_b.append(b)
            matr_r.append(row_r)
            matr_g.append(row_g)
            matr_b.append(row_b)

        r = checkByte(round(conv(matr_r, filter) / div))
        g = checkByte(round(conv(matr_g, filter) / div))
        b = checkByte(round(conv(matr_b, filter) / div))

        # r = checkByte(min(matr_r))
        # g = checkByte(min(matr_g))
        # b = checkByte(min(matr_b))
        
        
        # if r < 512:
        #     pixels2[i, j] = (255, 255, 255)
        # else:
        #     pixels2[i, j] = (0, 0, 0)
        
        
        pixels2[i, j] = (r, g, b)

plt.imshow(imFinal)
plt.show()

Попробуйте поменять фильтр и посмотреть, что будет.

---

Давайте немного потренируемся в вычислении размера результата применения свёртки к картинке.

### Задача 1

Вычислить размер результата после применения свёртки, $I$ - размеры входного изображения, $f$ - размеры фильтра:

1. $I = (50, 50, 3)$, $f = (3, 3)$, $stride = 1$.

   Вычисляем:
   
    * по ширине: $50 - 3 + 1 = 48$
    
    * по высоте: $50 - 3 + 1 = 48$
    
   то есть размер результата будет $(48, 48)$.


2. $I = (1024, 768, 3)$, $f = (5, 5)$, $stride = 2$.

   Вычисляем:

   <img src="../img/convnet_pooling_3.png" width=500>

   Из рисунка видно, что выходная ширина равна количеству нечётных чисел от $1$ до $1020$, т.е. $510$.

   Аналогично по высоте получаем $382$, то есть размер результата будет $(510, 382)$.


3. $I = (500, 700, 5)$, $f = (7, 4)$, $stride = 2$.
    
   Размер результата будет $(247, 349)$.


4. Выведите общую формулу для $I = (H, W, C)$, $f = (F, F)$ (фильтры обычно всё же квадратные).

   Общая формула $\displaystyle \frac{H - F}{stride} \times \frac {W - F}{stride} \times C$ (деление с округлением вверх?).
   

5. Теперь добавьте в получившуюся формулу $padding = p$:

   $\displaystyle \frac{H - F + 2p}{stride} \times \frac {W - F + 2p}{stride} \times C$

---

## Pooling

**Pooling (пулинг)** - операция, нужная для уменьшения размерности по ширине и по высоте. Можно брать очень много операций в качестве пулинга, например, минимум из элементов, максимум, среднее, сумму и т.д.

Обычно используется **max-pooling** и **avg-pooling**.

Пример применения max-pooling'а:

<img src="../img/convnet_conv_pool_3.jpg">

Примеры применения max-pooling'а и avg-pooling'а:

<img src="../img/convnet_conv_pool_4.png">

Если на вход подаётся изображение с несколькими каналами, то пулинг берётся по каналам, то есть если это цветная картинка $H \times W \times C$, и мы применяем к ней pooling $2 \times 2$, то получим на выходе картинку $\displaystyle\frac{H}{2} \times \frac{W}{2} \times C$.

Эта операция весьма простая, но лучше разобрать несколько примеров.

---

## Полезные ссылки

1. Примеры написания нейросетей на **PyTorch** (офийиальные туториалы на английском):
    
    * https://pytorch.org/tutorials/beginner/pytorch_with_examples.html#examples
    
    * https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

2. Один из самых подробных и полных курсов по deep learning на данный момент - это [курс Стэнфордского Университета](http://cs231n.github.io/)

3. Практически исчерпывающая информация по основам свёрточных нейросетей (из cs231n) (на английском):

    * http://cs231n.github.io/convolutional-networks/

    * http://cs231n.github.io/understanding-cnn/

    * http://cs231n.github.io/transfer-learning/

4. Видео о Computer Vision от Andrej Karpathy: https://www.youtube.com/watch?v=u6aEYuemt0M